In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
movies=pd.read_csv(r"C:\Users\mohad\Downloads\1636213079611301\movies.csv")
ratings=pd.read_csv(r"C:\Users\mohad\Downloads\1636213079611301\ratings.csv")

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies['year']=movies.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies['year']=movies.year.str.extract('(\d\d\d\d)',expand=False)
movies['title']=movies.title.str.replace('(\(\d\d\d\d\))','')
movies['title'] = movies['title'].apply(lambda x: x.strip())
movies.head(10)

C:\Users\mohad\AppData\Local\Temp\ipykernel_15408\3619945172.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['title']=movies.title.str.replace('(\(\d\d\d\d\))','')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995
5,6,Heat,Action|Crime|Thriller,1995
6,7,Sabrina,Comedy|Romance,1995
7,8,Tom and Huck,Adventure|Children,1995
8,9,Sudden Death,Action,1995
9,10,GoldenEye,Action|Adventure|Thriller,1995


#### we removed genres column because we wont decide based on contents and features of an item.

In [5]:
movies = movies.drop('genres', 1)
movies.head()

C:\Users\mohad\AppData\Local\Temp\ipykernel_15408\3037692081.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  movies = movies.drop('genres', 1)


,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


#### removing timestamp column because we dont need it in recommending process.

In [6]:
ratings = ratings.drop('timestamp', 1)
ratings.head()

C:\Users\mohad\AppData\Local\Temp\ipykernel_15408\1184197411.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ratings = ratings.drop('timestamp', 1)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [7]:
userInput=[
    {'title':'Heat','rating':8},
    {'title':'Jumanji','rating':5},
    {'title':'GoldenEye','rating':6.7},
    {'title':'Waiting to Exhale','rating':4.3},
    {'title':'Toy Story','rating':7.5}
]
inputMovies=pd.DataFrame(userInput)
inputMovies

,title,rating
0,Heat,8.0
1,Jumanji,5.0
2,GoldenEye,6.7
3,Waiting to Exhale,4.3
4,Toy Story,7.5


#### to add the movie id to inputMovies df, we search in movies df and take the ones which their name is in the list of the names of inputMovies df. and then we merge two dfs to add the movie id into inputMovies. the result is as kind of movies df so we have to drop  year column.

In [8]:
inputId = movies[movies['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop('year', 1)
inputMovies

C:\Users\mohad\AppData\Local\Temp\ipykernel_15408\492395849.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies = inputMovies.drop('year', 1)


,movieId,title,rating
0,1,Toy Story,7.5
1,2,Jumanji,5.0
2,4,Waiting to Exhale,4.3
3,6,Heat,8.0
4,10,GoldenEye,6.7


#### for collaborative fitering recommendation system we need to consider other users and their ratings.
#### so to find the subset of users who have watched and rated the input movies, we search into ratings df by id of the movies that our user have rated.

In [16]:
userSubset = ratings[ratings['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
0,1,1,4.0
2,1,6,4.0
516,5,1,4.0
560,6,2,4.0
562,6,4,3.0


#### we grouped the users by their ids. so the rows that have the same value on userID column will be in one group.

In [19]:
userSubsetGroup = userSubset.groupby(['userId'])
userSubsetGroup.get_group(6)

,userId,movieId,rating
560,6,2,4.0
562,6,4,3.0
564,6,6,4.0
567,6,10,3.0


#### then we sorted the user subset by the number of movies they have rated.
#### lambda function get the userSubsetGroup as parameter and sort the df by the length of the second column 'movieid'. it means the users that share the most movies in common with the input user will get on the top of df.

In [20]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
userSubsetGroup[:3]

C:\Users\mohad\AppData\Local\Temp\ipykernel_15408\3492080122.py:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)


[(6,
       userId  movieId  rating
  560       6        2     4.0
  562       6        4     3.0
  564       6        6     4.0
  567       6       10     3.0),
 (68,
         userId  movieId  rating
  10360      68        1     2.5
  10361      68        2     2.5
  10364      68        6     4.0
  10366      68       10     4.5),
 (82,
         userId  movieId  rating
  12730      82        1     2.5
  12731      82        2     3.0
  12732      82        6     3.5
  12733      82       10     3.5)]

#### we selected the top 100 users to work with. and we dont need the users that have less movies in common with input user.

In [21]:
userSubsetGroup = userSubsetGroup[:100]

#### our system is user-based so we have to find the similarity of selected users to input user.
#### we used pearson correlation which is an invariant to scaling.
#### pearson correlation coefficient 1 means that user is similar to input and -1 means opposite.
#### first we created an empty dictionary where key is user id and its value is similarity coefficient.
#### then we iterated through userSubsetGroup and sorted every group by movie id and also did the same for inputMovies. 
#### then we keep the length of each group which is the number of movies that current user has rated. and then we created a new df named temp_df  which is the movies both input and current user have in common. and then we converted the ratings of input and current user into a list.
#### now it's time to calculate the pearson correlation.
#### x is considered as input user and y as current user.
#### sxy is the sum of the product of the deviation of each rating from the mean of ratings.
#### sxx is the sum of squeres of deviation from mean for input user.
#### syy is the sum of squeres of deviation from mean for current user.
#### if denominators are not equal to zero we define the similarity coefficient as Sxy/sqrt(Sxx*Syy) and add it to pearsonCorrelationDict as the value otherwise we add 0.
#### the whole process will be repeat for each group.
#### and by the end we'll know how much each user is similar to input user.

In [23]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [24]:
pearsonCorrelationDict.items()

dict_items([(6, 0.3454442679267335), (68, 0.39375855813509264), (82, 0.10598118338654054), (91, 0.9060682839860661), (160, -0.3551424026022875), (202, 0.6088155473182103), (217, 0.26362486509824856), (219, 0.5281643011010205), (274, 0.9132233209773191), (353, 0.24854790640047897), (380, 0), (411, 0.6893059903861148), (414, 0.3551424026022899), (434, 0.9865570045844902), (470, 0.3551424026022899), (480, 0.4833122526801187), (559, 0.5299059169326952), (573, 0.3551424026022971), (590, 0.8668755838002766), (599, 0.7872396565780562), (18, 0.9332565252573836), (19, 0.31330417999518434), (21, 0.2034970342407617), (57, 0.13206763594884666), (84, 0.7689578149010815), (93, -0.9790755624849459), (112, 0.9332565252573836), (117, 0), (140, 0.42341515917871225), (144, 0.7461557922161149), (166, -0.5490856186527587), (182, 0.9912407071619476), (186, 0), (220, -0.1996674976919213), (226, 0.6657713824897588), (232, -0.6657713824897699), (240, -0.2034970342407634), (282, 0.6099942813304327), (288, 0.949

#### we converted the pearsonCorrelationDict to pandas data frame and by defining 'index' for orient, the keys in passed dictionary will become the rows of data frame.
#### we added the 'userId' and 'similarityIndex' as the columns.
#### the values of 'similarityIndex' are the values in dictionary and the values of 'userId' are the keys in dictionary.

In [25]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.345444,6
1,0.393759,68
2,0.105981,82
3,0.906068,91
4,-0.355142,160


#### we selected the top 50 of users that have the most similarityIndex. 

In [26]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
91,1.0,177
88,1.0,135
81,1.0,64
79,1.0,62
90,1.0,169


#### to create the weighted average of the ratings of the movies using pearson correlation as weight, we merged the topUsers with ratings to get the movie id and its rating
#### in fact we want to get the movies that the similar users have watched and rated to recommend to input user.

In [27]:
topUsersRating=topUsers.merge(ratings, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,1.0,177,1,5.0
1,1.0,177,2,3.5
2,1.0,177,7,1.0
3,1.0,177,11,3.0
4,1.0,177,16,3.0


#### we multiply similarityIndex as weight to rating and add the result in a new column named 'weightedRating'.

In [28]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,177,1,5.0,5.0
1,1.0,177,2,3.5,3.5
2,1.0,177,7,1.0,1.0
3,1.0,177,11,3.0,3.0
4,1.0,177,16,3.0,3.0


#### we created a temporary df to keep the sum of similarity and sum of the weighted ratings of each movie. in this way we can clearly see that how much each movie has been rated in total and how much is similar to the movies watched by the input user.

In [29]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,35.150948,144.426547
2,23.807514,73.134599
3,10.210233,34.536954
4,1.458264,3.685485
5,10.317148,32.202725


#### we created the recommendation data frame and divided the sum of weighted ratings by sum of similarity index to get the wighted average recom score.

In [34]:
recommendation = pd.DataFrame()
#Now we take the weighted average
recommendation['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation['movieId'] = tempTopUsersRating.index
recommendation.head()

,weighted average recommendation score,movieId
movieId,,
1,4.108753,1
2,3.071912,2
3,3.382582,3
4,2.527310,4
5,3.121282,5


#### in the final step we searched into movies df by the movie ids of recommended movies to find the information such as title and the release yaer of them.
#### and here are the top 10 movies we recommend to user.

In [32]:
movies.loc[movies['movieId'].isin(recommendation.head(10)['movieId'].tolist())]

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995
5,6,Heat,1995
6,7,Sabrina,1995
7,8,Tom and Huck,1995
8,9,Sudden Death,1995
9,10,GoldenEye,1995
